# Behavior analysis of free swimming pairs
## 81c:NTR and controls


In [ ]:
import socket
import os
import pandas as pd

#define folders based on hostName
hostName=socket.gethostname()

if hostName == 'O1-332': #work laptop
    codeDir = 'C:\\Users\\jlarsch\\Documents\\jlsocialbehavior'
    
elif hostName == 'O1-298'or (hostName == 'O1-339'): #work desktop
    codeDir = 'C:\\Users\\jlarsch\\Documents\\jlsocialbehavior'
else:
    print('No folders defined for this computer...')

if 'NB_path' not in locals():
    NB_path = os.getcwd()
else:
    os.chdir(NB_path)
    
propsFn='props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
artDir=props['ArtDir']
outputDir = props['outputDir']
RawDataDir = props['RawDataDir']+'behavior\\'
ProcessingDir = props['ProcessingDir']
outputDir = props['outputDir']
FigureDir=props['FigureDir']

expFile=props['allExpBehFn']
anFile=props['allAnBehFn']


props
print('Notebook was last executed on computer: '+hostName)
print('baseFolder = ' + base)



os.chdir(codeDir)
props

In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import fnmatch
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime
from scipy import stats
import statsmodels.stats.api as sms

import functions.matrixUtilities_joh as mu
import functions.notebookHelper as nh
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
#import functions.peakdet as pkd
#import functions.metaTree as mt


pfp.paper()
plt.rcParams['svg.fonttype'] = 'none'
inToCm=2.54

In [ ]:
expFile

In [ ]:
info=pd.read_excel(expFile)
ix=(info.stimulusProtocol=='81cNTR_free')#&(info.date.isin(['2019-08-05','2019-08-06','2019-09-25','2019-10-04']))
info=info[ix].reset_index(drop=True)
info

In [ ]:
infoAn=pd.read_excel(anFile)
infoAn.tail()

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
birthDayAll=[]
anIDsAll=[]
camHeightAll=[]

camHeight=[105,180] #for arena up,dn

for index,row in info.iterrows():

    startDir=RawDataDir+row.path+'\\'
    if not os.path.isdir(startDir):
        startDir=RawAlt+row.path+'\\'
    if not os.path.isdir(startDir):
        startDir=RawAlt2+row.path+'\\'
        #print(index,startDir)
    
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
    camHeightAll.append(camHeight[('_dn_' in head)*1])
    
    anNrs=row.anNr #Note that anNrs are 1 based!
    if ':' in anNrs:
        a,b=anNrs.split(sep=':')
        anNrs=np.arange(int(a),int(b)+1)
    else:
        anNrs=np.array(anNrs.split()).astype(int)
        
    anIDs=anNrs #-1 no more 0-based since using pandas merge to find animal numbers
    anIDsAll.extend(anIDs)

    bd=infoAn[infoAn.anNr.isin(anIDs)].bd.values
    #bd=infoAn.bd.values[anIDs-1] #a bit dirty to use anIDs directly here. Should merge
    birthDayAll.append(' '.join(list(bd)))

info['camHeight']=camHeightAll
info['txtPath']=posPath
info['pairList']=PLPath
info['aviPath']='default'
info['birthDayAll']=birthDayAll
info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 120 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 1 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
#info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir
info['expTime']=expTime
info['readLim'] = 54*5*60*30+11

In [ ]:
csvFile=os.path.join(ProcessingDir,'processingSettings.csv')
info.to_csv(csvFile,encoding='utf-8')
info.tail()

In [ ]:
info.shape

In [ ]:
rereadData=1
if rereadData:
    def readExperiment(keepData=True):
        tmp=es.experiment_set(csvFile=csvFile,MissingOnly=True)
        if keepData:
            return tmp
        else:
            return 1

    expSet=readExperiment(keepData=False)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*35)
    tmp.animalIndex=np.array(anIDsAll)[tmp.animalIndex]
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]
df=pd.merge(df,infoAn[['anNr','line','group']],left_on='animalIndex',right_on='anNr',how='left')
df=pd.merge(df,info[['date']],left_on='animalSet',right_on=info.index,how='left')

print('df shape',df.shape)
df.tail()

## Habituation or Fatigue within 20 hours?

In [ ]:
def sem(x):
    return np.std(x,axis=0)/np.sqrt(x.shape)

x=np.random.random(100)

def ci95(x):
    return np.nanmean(x)-sms.DescrStatsW(x[np.isfinite(x)]).tconfint_mean()[0]

print('std of uniform = 0.2886751345948129. STDdata:',np.std(x), 'semData:',sem(x),'samples:',x.shape) 
print('ci95:',ci95(x))


Plot shoaling index during closed loop skype episodes over time.

In [ ]:
dfPlot=(df.groupby(['inDishTime','group']).si.agg(['mean','std',sem,ci95])
    .unstack()
    .stack(dropna=True)
    .reset_index())

dfPlot.head()


In [ ]:
fig, axes = plt.subplots(figsize=(6, 3))
g=sns.scatterplot(data=dfPlot,x='inDishTime',hue='group',y='mean')
plt.xlim([0,5.5*60])
plt.ylim([-.05,.6])
plt.xlabel('Time (Minutes)')
plt.ylabel('Attraction')
plt.title('Mean attraction, all animals')
#plt.legend(title='Stimulus dot motion')
#plt.legend(labels=['Continuous motion','Bout motion','Continuous + loom','Bout + loom'],
#          title='Stimulus dot motion')

#new_title = 'Stimulus dot motion'
#g.legend_.set_title(new_title)
# replace labels
plt.legend(ncol=1,handletextpad=0,bbox_to_anchor=(1, 1),loc='upper left')


sns.despine()

figPath=base+'ScreenTimeCourse.png'
#plt.savefig(figPath,bbox_inches='tight')


In [ ]:

ageSet=info.set.values
df['ageSet']=ageSet[df.animalSet]

setup=info.path.values


setup=np.array([x[22:24] if len(x) > 16 else x[-3:-1] for x in setup])
df['setup']=setup[df.animalSet]


df.head()


# Mean response over all stimuli per animal

In [ ]:
df.animalSet.unique()

In [ ]:
sns.set_palette('viridis',3)
co=sns.color_palette("viridis", 3)
idx=(df['inDishTime']<60) & (df['inDishTime']>0)&(df.episode=='conspecific')&(df.group!='none')
dfDR=df[idx]
dfEpiAn=dfDR.groupby(['episode','animalIndex','line','setup','group','ageSet','date'],sort=True).mean().reset_index()

In [ ]:
sns.stripplot(data=dfEpiAn,x='group',y='si',zorder=-1,hue='group')
sns.pointplot(data=dfEpiAn,x='group',y='si',hue='line',zorder=100,scale=0.2,palette=['gray'])
sns.pointplot(data=dfEpiAn,x='group',y='si',join=False,zorder=100)
ax=plt.gca()
ax.legend_.remove()

In [ ]:
dfEpiAn['lineSet']=[x+'_'+y for x,y in zip(dfEpiAn.line, dfEpiAn.ageSet)]
dfEpiAn['lineSetAge']=[x+'_'+y+'_'+z for x,y,z in zip(dfEpiAn.line, dfEpiAn.ageSet, dfEpiAn.age.astype('str'))]

In [ ]:
tmp.reset_index()

In [ ]:
a

In [ ]:
tmp=dfEpiAn.groupby(['animalIndex','group']).si.mean().reset_index()
ix=tmp.animalIndex%2==0
a=tmp[(tmp.group=='abl')&ix].si
b=tmp[(tmp.group=='ctr')&ix].si
s,p=stats.ttest_ind(a,b)
(s,p)

In [ ]:
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

In [ ]:
d=cohen_d(a,b)
d

In [ ]:
plt.rcParams['svg.fonttype'] = 'none'
fig,axes = plt.subplots(figsize=(4/inToCm,10/inToCm))
sns.pointplot(data=dfEpiAn,
              x='group',
              y='si',
              #hue='group',
              order=['ctr','abl'],
              zorder=100,
              scale=1,
              join=False,
             ci='sd',
              
              color='k',
            # dodge=.5
             )

sns.stripplot(data=dfEpiAn,
              x='group',
              order=['ctr','abl'],
              hue_order=['ctr','abl'],
              y='si',
              hue='group',
              size=4,
              zorder=-100,
              palette=['k','r'],
              alpha=0.5,
              label=None,
             #dodge=.5
             )

sns.despine()

axes.set_ylim(-.2,.8)
axes.set_ylabel('Attraction')
axes.axhline(0,ls=':',color='k')
#axes.set_title('Attraction per group');
l=axes.legend()
h,l = axes.get_legend_handles_labels()
axes.legend(h, ['Abl','Ctr'][::-1],title='',ncol=1,handletextpad=0,bbox_to_anchor=(.5, .8),
          frameon=False)

axes.set_xlabel('')
axes.set_xticks([])
#plt.legend().remove()
axes.text(.5,.75,'***')
plt.ylim([-.2, .85])
#plt.xticks(rotation=90);
plt.title('Conspecific')
figPath=FigureDir+'5_MTZ_ConspecificShoaling.svg'
#plt.savefig(figPath,bbox_inches='tight')

In [ ]:
plt.rcParams['svg.fonttype'] = 'none'
fig,axes = plt.subplots(figsize=(1.8/inToCm,4/inToCm))
sns.pointplot(data=dfEpiAn,
              x='group',
              y='si',
              #hue='group',
              order=['ctr','abl'],
              zorder=100,
              scale=1,
              join=False,
             ci='sd',
              
              color='k',
            # dodge=.5
             )

sns.stripplot(data=dfEpiAn,
              x='group',
              order=['ctr','abl'],
              hue_order=['ctr','abl'],
              y='si',
              hue='group',
              size=4,
              zorder=-100,
              palette=['k','r'],
              alpha=0.5,
              label=None,
             #dodge=.5
             )

sns.despine()

axes.set_ylim(-.2,.8)
axes.set_ylabel('Attraction')
axes.axhline(0,ls=':',color='k')
#axes.set_title('Attraction per group');
l=axes.legend()
h,l = axes.get_legend_handles_labels()
axes.legend(h, ['Abl','Ctr'][::-1],title='',ncol=1,handletextpad=0,bbox_to_anchor=(.5, .8),
          frameon=False)

axes.set_xlabel('')
axes.set_xticks([])
#plt.legend().remove()
axes.text(.1,.85,'p=5e-08')
axes.text(.1,.73,'d=-5.7')
plt.ylim([-.2, .9])
#plt.xticks(rotation=90);
plt.title('Conspecific')
figPath=FigureDir+'5_MTZ_ConspecificShoaling.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
#Note: each animal is plotted here, not pairs.

dfEpiAn[['animalIndex','line','group','si']].to_csv(FigureDir+'SourceDataS7d.csv')
dfEpiAn[['animalIndex','line','group','si']]

In [ ]:
h

In [ ]:
fig, axes = plt.subplots(figsize=(2, 3))
ix=(dfEpiAn.episode=='02k20f')#&(~dfEpiAn.lineSet.str.contains('_2_'))

selDat=dfEpiAn

allCat=selDat.lineSet.unique()
allCat.sort()
allCat=allCat[::-1]

sns.swarmplot(data=selDat,
              x='lineSetAge',
              y='si',
              hue='group',
              zorder=-1,
              dodge=.5,
              size=5,
              alpha=.5,
              #order=allCat,
              #hue_order=["hi", "lo"],
             )

sns.pointplot(data=selDat,
              x='lineSetAge',
              y='si',
              hue='group',
              zorder=1000,
              dodge=.5,
              join=False,
              color='k',
              #order=allCat,
              ci='sd',
              legend=None,
              #hue_order=["hi", "lo"],
             )


sns.despine()

axes.set_ylabel('Attraction')
axes.set_xticklabels('')
axes.set_xticks([])
axes.set_xlabel('')


axes.axhline(0,ls=':',color='k')
#axes.set_title('Selection F1');

#plt.xticks(rotation=90);

handles, labels = axes.get_legend_handles_labels()

l = plt.legend(handles[0:4], labels[0:4], title='Stimulus',ncol=1,handletextpad=0,
               bbox_to_anchor=(1, 1.05),
              frameon=False)

#figPath=base+'SelectionAllToF1.png'
#plt.savefig(figPath,bbox_inches='tight')

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
import matplotlib as mtp
mtp.__version__

In [ ]:
import scipy as scp
scp.__version__